## 1D Ising Model with Transverse Field

### Hamiltonian:

The Hamiltonian of the system is:

$$H = - \sum_{n=1}^{L-1} \sigma^z_n \sigma^z_{n+1} - \sum_{n=1}^{L} g_n\sigma^x_n$$

where:
- $L$ is the number of spin-1/2 particles.
- $g$ is the transverse magnetic field strength.
- $\sigma^x_n$, $\sigma^z_n$ are spin-1/2 operators at site $n$ in $\hat{x}, \hat{z}$ direction.

### Phases:

The model has a second-order phase transition at the critical point $g_c = \pm 1$.

**Paramagnetic Phase**:

The model is in a **paramagnetic phase**, when $|g| \gg 1$. In this case the ground state of the system has no degeneracy and it is a state in which all spins are pointing along $\hat{x}$:

$$\ket{GS}=\ket{\rightarrow\rightarrow\rightarrow\cdots\rightarrow\rightarrow\rightarrow}$$

On the other hand, the most excited state in this phase would be:

$$\ket{\leftarrow\leftarrow\leftarrow\cdots\leftarrow\leftarrow\leftarrow}$$

**Ferromagnetic Phase**:

The model is in a **ferromagnetic phase**, when $|g| \ll 1$. A ground state of the system has a two-fold degeneracy and it can be one of the following states (or their superposition):

$$\ket{\uparrow\uparrow\uparrow\cdots\uparrow\uparrow\uparrow}, \quad \ket{\downarrow\downarrow\downarrow\cdots\downarrow\downarrow\downarrow}$$

The most excited state is spanned by:

$$\ket{\uparrow\downarrow\uparrow\cdots\uparrow\downarrow\uparrow}, \quad \ket{\downarrow\uparrow\downarrow\cdots\downarrow\uparrow\downarrow}$$

## Simulations

Our goal is to simulate a quench of a finite Ising chain with size up to $1000$ spins. A *quench* refers to a change in time of a control parameter $g$, which can be either uniform or spatially dependent.

Exponential growth of the Hilbert space with the number of particles makes exact diagonalization methods inefficient to simulate a system of as few as $50$ particles. \
However, this model can be mapped to a system of non-interacting spinless fermions using the Jordan-Wigner transformation, resulting in a quadratic form that can be diagonalized via the Bogoliubov transformation.\
 This approach allows us to work with $2L \times 2L$ matrices, instead of $2^L \times 2^L$ ones.

The Jordan-Wigner transformation is defined as:
$$\sigma^x_n = 1 - 2 c^{\dagger}_n c_n, \quad \sigma^z_n = -(c_n + c^{\dagger}_n) \prod_{m<n} (1 - 2 c^{\dagger}_m c_m)$$

The hamiltonian becomes quadratic in J-W fermions:
$$H = \sum_{n=1}^{L} g_n \left( 2 c^{\dagger}_n c_n - 1 \right) - 2 \sum_{n=1}^{L-1} \left(c^{\dagger}_n c^{\dagger}_{n+1} + c^{\dagger}_n c_{n+1} + c^{\dagger}_{n+1} c_{n} + c_{n+1} c_n \right)$$

This quadratic hamiltonian can be diagonalized:
$$H = \sum_{m=1}^{L}\omega_m \left(\gamma_m^{\dagger} \gamma_m - \frac{1}{2} \right)$$

via the Bogoliubov transformation
$$c_n = \sum_{m=1}^{L} ( u_{n,m} \gamma_m + v_{n,m}^* \gamma^{\dagger}_m)$$

where $u^{\pm}_{n,m} = \left(u_{n,m} \pm v_{n,m} \right) / \sqrt{2}$ are positive eigenmodes ($\omega_m \geq 0$) of the stationary Bogoliubov-de-Gennes (BdG) equations:
$$\omega_m u^{\pm}_{n,m} = 2 g_n u^{\mp}_{n,m} - 2 u^{\mp}_{n\mp1, m}$$

To simulate the evolution of BdG eigenmodes, we use the time-dependent version of BdG equation:
$$i \hbar \partial_{t} u^{\pm}_{n,m} = 2 g_n(t) u^{\mp}_{n,m} - 2 u^{\mp}_{n\mp1, m}$$

We impose OBC (though the code also has an option to simulate a system with PBC): 
$$u^{\pm}_{L+1}=u^{\pm}_{0}=0$$

The time-dependent BdG eq. can be rewritten in the vectorized form:
$$i \hbar \partial_t \begin{pmatrix} \vec{u}^{+} \\ \vec{u}^{-} \end{pmatrix} = H_1(t)\begin{pmatrix} \vec{u}^{+} \\ \vec{u}^{-} \end{pmatrix}+H_2 \begin{pmatrix} \vec{u}^{+} \\ \vec{u}^{-} \end{pmatrix}$$

Formal solution to the equation above would involve the Dyson series, as hamiltonians at different times do not commute.

Instead, we can divide the quench time $\tau_Q$ into $N$ time steps of size $dt = \tau_Q / N$, and assume that the $H$ is constant at each step with $g=g(ndt+dt/2)$:

$$U(t) \approx \prod_{n=0}^{N-1} e^{-i \hbar dt (H_1(ndt+dt/2) + H_2)}$$

As we will see, it is possible to analytically diagonalize both $H_1$ and $H_2$ separately, rather than their sum $H_1+H_2$. Therefore, it is advantageous to break this exponential operator into three parts using the third order Suzuki-Trotter decomposition:

$$e^{-i \hbar dt (H_1(ndt+dt/2) + H_2)} = e^{-i \hbar \frac{dt}{2} H_2} e^{-i \hbar dt H_1(ndt+dt/2)} e^{-i \hbar \frac{dt}{2} H_2} + O(dt^3)$$

$H_1$ is diagonal in the ${u,v}$ basis:

$$i \hbar \partial_t u_{n,m} = 2g_n(t) u_{n,m} + \dots$$
$$i \hbar \partial_t v_{n,m} = -2g_n(t) v_{n,m} + \dots$$

and $H_2$ is diagonal in $\tilde{u}^{\pm}_{n,m}$ basis

$$i \hbar \partial_t \tilde{u}^+_{n,m} = \dots - 2\tilde{u}^+_{n,m}$$
$$i \hbar \partial_t \tilde{u}^-_{n,m} = \dots + 2\tilde{u}^-_{n,m}$$

where:

$$\tilde{u}^+_n = \frac{u^{+}_n + u^-_{n-1}}{\sqrt{2}}, \quad \tilde{u}^-_n = \frac{u^{+}_{n+1} - u^-_{n}}{\sqrt{2}}$$

Density of excitations can be found as an expectation value of the number of Bogoliubov quasiparticles in the final state:
$$p = \frac{1}{N}\sum_{m}\bra{\psi(t_f)}\gamma^{\dagger}_m\gamma_m \ket{\psi(t_f)}$$

where Bogoliubov quasiparticles are defined by the eigenmodes of stationary BdG eqs. at the final point:
$$\gamma_m = \sum_{n=1}^{N} ( u^*_{n,m} c_n + v^*_{n,m} c^{\dagger}_n )$$
and fermionic operators are defined by evolved modes:
$$c_n = \sum_{m=1}^{N} ( u_{n,m}(t_f) \tilde{\gamma}_m + v_{n,m}^*(t_f) \tilde{\gamma}^{\dagger}_m )$$

with a constraint that $\tilde{\gamma}_m \ket{\psi(t_f)}=0$

This was a general idea on how to efficiently simulate dynamics of Ising model with transverse magnetic field.

## Kibble-Zurek Mechanism
**Main idea**: Spontaneous symmetry breaking leads to the formation of topological defects.

In Ising model we have the following SSB pattern:
$$\mathbb{Z}_2 \rightarrow \mathbb{1}$$

Now, suppose that we change our control parameter over some time scale $\tau_Q$, and near the critical point it behaves as:
$$g \sim \left(\frac{t}{\tau_Q}\right)^r$$

The question could be, for example, what is the size of domains, or what is the density of these topological defects? Answer:
$$d \sim \tau_Q^{-\frac{r\nu}{rz\nu+1}}$$

where $z, \nu$ are so-called critical exponents, and $r$ depends on the rate of change at which we
approach the critical point. For Ising model $z=\nu=1$ and in case of a sinusoidal ramp, which approaches
the critical point as quadratic function ($r=2$) we have:

$$d \sim \tau_Q^{-2/3}$$

In [54]:
from IPython.display import SVG, HTML, display

display(HTML('<div style="width:200px; height:300px;">' + SVG("../plots/KZ_scaling_plot.svg").data + '</div>'))

### Example 1
We start at $g=2$ and go down to the critical point $g_c=1$. We choose sine protocol:

$$g(t) =
\begin{cases}
2, & \text{if } \frac{t}{\tau_Q} < -\frac{\pi}{2} \\
3/2 - 1/2 \sin(\frac{t}{\tau_Q}), & \text{if } -\frac{\pi}{2} \leq \frac{t}{\tau_Q} \leq \frac{\pi}{2} \\
1, & \text{if } \frac{t}{\tau_Q} > \frac{\pi}{2}
\end{cases}$$

Data for $L=100, 200, 300, 400, 500, 1000$ confirm power-law scaling.

### Example 2
Dispersion relation for infinite 1D Ising model has the form:

$$\omega_k=2\sqrt{(g-\cos{k})^2+\sin^2{k}}$$

one can expand it near the Fermi point at $k=0$ for small $k$:

$$\omega_k \approx 2k \Rightarrow c=\frac{\mathrm{d} \omega}{\mathrm{d}k}=2$$

we see that there is a characteristic velocity in the system. We can think of it as a maximal group velocity at which information can propagate.

**Idea**: If a speed of ramp $g(t, n)$ moves slower than $c$, then those parts of the system that cross the critical point earlier may be able to communicate their choice of orientation of the order parameter to the parts that cross the transition later and bias them to make the same choice. Hence, smaller excitation density, and a better approximation of a final GS.

In [47]:
from IPython.display import IFrame

IFrame("../plots/inhomo_test_lin_plot_1000.pdf", width=800, height=500)